# 3.- Azure ML Resources

In [30]:
import yaml

from azure.identity import DefaultAzureCredential

from azure.mgmt.resource import ResourceManagementClient

from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError, HttpResponseError

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.ai.ml.entities import Environment

from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt.containerregistry.models import Registry, Sku, RunRequest

## Define Variables

In [31]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [32]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]
location = config["azure"]["location"]
acr_name = config["azure"]["acr_name"]
sku = config["azure"]["sku"]

## Azure Authentication

In [4]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

## Resource Group

In [5]:
# Initialize the Resource Management client
resource_client = ResourceManagementClient(credential, subscription_id)

In [6]:
def create_resource_group(resource_client, resource_group_name, location):
    try:
        # Attempt to get the resource group
        resource_group = resource_client.resource_groups.get(resource_group_name)
        print(f"Resource Group '{resource_group_name}' already exists in '{resource_group.location}'.")
    except ResourceNotFoundError:
        # If the resource group does not exist, create it
        resource_group_params = {"location": location}
        resource_group = resource_client.resource_groups.create_or_update(
            resource_group_name,
            resource_group_params
        )
        print(f"Resource Group '{resource_group_name}' created in '{resource_group.location}'.")
    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {e}")


In [ ]:
# Call the function to create the Resource Group
create_resource_group(resource_client, resource_group_name, location)

## Workspace

In [8]:
ml_client = MLClient(credential, subscription_id, resource_group_name)

In [9]:
def create_workspace(ml_client, workspace_name, location):
    try:
        # Intentar obtener el Workspace existente
        workspace = ml_client.workspaces.get(workspace_name)
        print(f"Workspace '{workspace_name}' ya existe en '{workspace.location}'.")
    except ResourceNotFoundError:
        # Si el Workspace no existe, crearlo de manera asíncrona
        workspace_poller = ml_client.workspaces.begin_create(
            Workspace(
                name=workspace_name,
                location=location  # Usa la variable 'location'
            )
        )
        workspace = workspace_poller.result()  # Espera a que la operación termine
        print(f"Workspace '{workspace_name}' creado en '{workspace.location}'.")
    except Exception as e:
        # Manejar otras excepciones
        print(f"Ocurrió un error: {e}")

In [27]:
create_workspace(ml_client, workspace_name, location)

## Azure Container Registry

In [18]:
acr_client = ContainerRegistryManagementClient(credential, subscription_id)

In [25]:
def create_container_registry(acr_client, registry_name, resource_group_name, location, sku):
    try:
        # Attempt to get the existing Container Registry
        registry = acr_client.registries.get(resource_group_name, registry_name)
        print(f"Container Registry '{registry_name}' already exists in '{registry.location}'.")
        return registry
    except ResourceNotFoundError:
        try:
            # Define the parameters for the Container Registry
            registry_params = Registry(
                location=location,
                sku=Sku(name=sku)
            )
            
            # Begin creating the Container Registry
            registry_poller = acr_client.registries.begin_create(
                resource_group_name=resource_group_name,
                registry_name=registry_name,
                registry=registry_params
            )
            
            # Wait for the operation to complete
            registry = registry_poller.result()
            print(f"Container Registry '{registry_name}' created successfully in '{registry.location}'.")
            return registry
        except ResourceExistsError:
            print(f"The Container Registry '{registry_name}' already exists.")
        except Exception as e:
            print(f"An error occurred while creating the Container Registry: {e}")

In [ ]:
# Create the Container Registry
registry = create_container_registry(acr_client, acr_name, resource_group_name, location, sku)